In [13]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import cftime
import glob
import warnings
xr.set_options(keep_attrs=True)
warnings.filterwarnings('ignore')
import sys
sys.path.insert(1, '../')
import my_utils as myf
import importlib
importlib.reload(myf)

<module 'my_utils' from '/gpfsm/dhome/laroach1/analysis/analysis_modelE-so-sst_pub/processing/../my_utils.py'>

In [2]:
c6dir = myf.cmip_dir+'CMIP/NASA-GISS/GISS-E2-1-G/historical/'
sspdir = myf.cmip_dir+'ScenarioMIP/NASA-GISS/GISS-E2-1-G/ssp245/'

In [3]:
griddir = myf.cmip_dir+'CMIP/NASA-GISS/GISS-E2-1-G/piControl/r1i1p1f1/fx/'
garea = xr.open_dataset(griddir+'areacella/gn/v20180824/areacella_fx_GISS-E2-1-G_piControl_r1i1p1f1_gn.nc').areacella
ocnfr = xr.open_dataset(griddir+'sftlf/gn/v20180824/sftlf_fx_GISS-E2-1-G_piControl_r1i1p1f1_gn.nc').sftlf
ocnfr = ocnfr/100.
ocnfr = 1. - ocnfr
garea = garea*ocnfr

In [4]:
def open_cmip(ens,var):
    
    ds = xr.open_mfdataset(c6dir+ens+'/Amon/'+var+'/gn/*/'+var+'_Amon_GISS-E2-1-G_historical_'+ens+'_*.nc')
    
    filestr = 'historical'
    if os.path.isdir(sspdir+ens):
        print('extend with SSP245')
        ds2 = xr.open_mfdataset(sspdir+ens+'/Amon/'+var+'/gn/*/'+var+'_Amon_GISS-E2-1-G_ssp245_'+ens+'_*.nc')
        ds = xr.concat([ds,ds2],dim='time').sel(time=slice('1979','2021'))
        filestr = 'hist+ssp245'
    ds['time'] = ds.indexes['time'].to_datetimeindex()
    ds['names'] = ens
    ds = ds.set_coords('names')
    ds = ds.sel(time=slice('1979','2021'))

    return ds, filestr

In [5]:
def get_wind_avg(ds, outfile):
          
    y1 = str(ds.time.values[0])[:4]
    y2 = str(ds.time.values[-1])[:4]
  
    mask = garea.copy()
    mask = mask.where(mask.lat<-50).where(mask.lat>-65)
    ds = (ds*mask).sum(dim=('lat','lon'))/mask.sum(dim=('lat','lon'))
    ds['region'] = '50-65S'
    ds = ds.set_coords('region')
    

    if 'type' in ds:
        ds = ds.drop('type')
    ds.to_netcdf(myf.processed_dir+'/timeseries/wind/'+outfile+str(y1)+'-'+str(y2)+'.nc')
    
    return ds

In [6]:
runmap = pd.read_csv('../e2.1runmap.csv',sep=',')
myens = runmap['names'].values

In [7]:
for ens in myens:
    ds1, filestr = open_cmip(ens, 'uas')
    ds2, filestr = open_cmip(ens, 'vas')
    ds = xr.merge([ds1,ds2])
    ds['ws'] = np.sqrt(ds['uas']**2+ds['vas']**2)
    ds = get_wind_avg(ds,'wind_avg_Amon_GISS-E2-1-G_'+filestr+'_'+ens+'_gn_')

extend with SSP245
extend with SSP245
extend with SSP245
extend with SSP245
extend with SSP245
extend with SSP245
extend with SSP245
extend with SSP245
extend with SSP245
extend with SSP245
extend with SSP245
extend with SSP245
extend with SSP245
extend with SSP245
extend with SSP245
extend with SSP245
extend with SSP245
extend with SSP245
extend with SSP245
extend with SSP245
extend with SSP245
extend with SSP245


In [14]:
merradir = '/discover/nobackup/projects/gmao/merra2/data/products/'
myfiles = sorted(glob.glob(merradir+'MERRA2_all/*/*/*tavgM_2d_slv_*.nc4'))
ds = xr.open_dataset(myfiles[0])
garea, _, _ = myf.grid_area_regll(ds.lat,ds.lon)
garea = xr.DataArray(garea,dims=('lat','lon'),coords = ds.isel(time=0).coords)
mask = garea.copy()
mask = mask.where(mask.lat<-50).where(mask.lat>-65)

y1, y2 = 1980, 2021
listds = []
for f in myfiles:
    ds = xr.open_dataset(f)[['U10M','V10M']]
    ds = ds.rename({'U10M':'uas','V10M':'vas'})
    ds['ws'] = np.sqrt(ds['uas']**2+ds['vas']**2)
    ds = (ds*mask).sum(dim=('lat','lon'))/mask.sum(dim=('lat','lon'))
    listds.append(ds)
ds = xr.concat(listds,dim='time').sel(time=slice(str(y1),str(y2)))
ds['region'] = '50-65S'
ds = ds.set_coords('region')
ds['names'] = 'OBS-MERRA2'
ds = ds.set_coords('names')

ds.to_netcdf(myf.processed_dir+'/timeseries/wind/wind_avg_Amon_MERRA2_'+str(y1)+'-'+str(y2)+'.nc')

In [11]:
ds = xr.open_dataset(myf.obs_dir+'/ERA-Interim/ei.moda.an.sfc.regn128sc.1979-2018_ptuv.nc')
ds = ds.rename({'latitude':'lat','longitude':'lon'})
garea, _, _ = myf.grid_area_regll(ds.lat,ds.lon)
garea = -xr.DataArray(garea,dims=('lat','lon'),coords = ds.isel(time=0).coords)
mask = garea.copy()
mask = mask.where(mask.lat<-50).where(mask.lat>-65)
ds = ds.rename({'u10':'uas','v10':'vas'})
ds['ws'] = np.sqrt(ds['uas']**2+ds['vas']**2)
ds = (ds*mask).sum(dim=('lat','lon'))/mask.sum(dim=('lat','lon'))
ds['region'] = '50-65S'
ds = ds.set_coords('region')
ds['names'] = 'OBS-ERA-Interim'
ds = ds.set_coords('names')
ds.to_netcdf(myf.processed_dir+'/timeseries/wind/wind_avg_Amon_ERA-Interim_1979-2018.nc')